# Dreambooth
### Notebook implementation by Joe Penna (@MysteryGuitarM on Twitter) - Improvements by David Bielejeski
Latest information on:
https://github.com/JoePenna/Dreambooth-Stable-Diffusion

## Build Environment

In [ ]:
# BUILD ENV
from IPython.display import clear_output

try:
 !pip install omegaconf
 !pip install einops
 !pip install pytorch-lightning==1.6.5
 !pip install test-tube
 !pip install transformers
 !pip install kornia
 !pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
 !pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
 !pip install setuptools==59.5.0
 !pip install pillow==9.0.1
 !pip install torchmetrics==0.6.0
 !pip install -e .
 !pip install protobuf==3.20.1
 !pip install gdown
 !pip install -qq diffusers["training"]==0.3.0 transformers ftfy
 !pip install huggingface_hub
 !pip install captionizer==1.0.1
 !pip install ipywidgets

 clear_output()
 print("✅ Dependencies successfully installed")

except:
 print("❌ Error installing dependencies")



# Download the 1.5 model from hugging face
You can also provide your own v1.* model for training by uploading it and renaming it to "model.ckpt".  It should be in the same directory as dreambooth_runpod_joepenna.ipynb

In [ ]:
# Download the 1.5 sd model
from IPython.display import clear_output
from huggingface_hub import hf_hub_download

downloaded_model_path = hf_hub_download(
 repo_id="panopstor/EveryDream",
 filename="sd_v1-5_vae.ckpt"
)

# Move the sd_v1-5_vae.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")

## Dreambooth Training Environment Setup


In [ ]:
%run juptyer-notebook-helpers/create-config-form-inputs.ipynb

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

In [ ]:
from IPython.display import clear_output
import os
import sys
import json
import shutil

clear_output()

# parse the config file "joepenna-dreambooth-configs/active-config.json"
config_file_path = "./joepenna-dreambooth-configs/active-config.json";
if not os.path.exists(config_file_path):
 print(f"{config_file_path} not found.", file=sys.stderr)
else:
 config_file = open(config_file_path)
 config_parsed = json.load(config_file)
 print("✅ Loaded the config file, proceeding to training")
 print()
 print("active-config.json")
 print("------------------")
 print(json.dumps(config_parsed, indent=4))

 reg_data_root = "/workspace/Dreambooth-Stable-Diffusion/regularization_images/" + config_parsed.dataset

 leftover_training_files = ["./training_images/.ipynb_checkpoints", "./regularization_images/.ipynb_checkpoints"]
 for i, ipynb_checkpoint_folder in enumerate(leftover_training_files):
  if os.path.exists(ipynb_checkpoint_folder):
   shutil.rmtree(ipynb_checkpoint_folder)

 !python "main.py" \
     --base "configs/stable-diffusion/v1-finetune_unfrozen.yaml" \
     -t \
     --actual_resume "model.ckpt" \
     --reg_data_root "{reg_data_root}" \
     -n "{config_parsed.project_name}" \
     --gpus 0, \
     --data_root "/workspace/Dreambooth-Stable-Diffusion/training_images" \
     --max_training_steps {config_parsed.max_training_steps} \
     --class_word "{config_parsed.class_word}" \
     --token "{config_parsed.token}" \
     --no-test \
     --flip_p {config.flip_percent} \
     --learning_rate {config_parsed.learning_rate} \
     --save_every_x_steps {config_parsed.save_every_x_steps}

## Copy and name the checkpoint file(s)

In [ ]:
import re
import shutil
import glob
from datetime import datetime, timezone

def create_checkpoint_file_name(
        config_parsed,
        date,
        training_images_count,
        training_steps,
):
 return f"{date}_{config_parsed.project_name}_" \
        f"{training_steps}_steps_" \
        f"{training_images_count}_training_images_" \
        f"{config_parsed.token}_token_" \
        f"{config_parsed.class_word}_class_word.ckpt".replace(" ", "_")

# parse the config file "joepenna-dreambooth-configs/active-config.json"
config_file_path = "./joepenna-dreambooth-configs/active-config.json";
if not os.path.exists(config_file_path):
 print(f"{config_file_path} not found.", file=sys.stderr)
else:

 config_file = open(config_file_path)
 config_parsed = json.load(config_file)
 training_images_count = str(len(glob.glob("./training_images/*")))
 date_string = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H-%M-%S")

 trained_models_path = "./trained_models"
 if not os.path.exists(trained_models_path):
  os.mkdir(trained_models_path)

 # copy the config file to the model directory as well
 shutil.copy(config_file_path, f"{trained_models_path}/{config_parsed.config_file_name}")

 latest_training_directory = glob.glob("./logs/*")[-1]


 if config_parsed.save_every_x_steps == 0:
  # Copy the checkpoint into our `trained_models` folder
  last_checkpoint_file = f"{latest_training_directory}/checkpoints/last.ckpt"
  file_name = create_checkpoint_file_name(
   config_parsed = config_parsed,
   date = date_string[-1],
   training_images_count = training_images_count,
   training_steps = str(config_parsed.max_training_steps),
  )

  # Move the checkpoint
  shutil.move(last_checkpoint_file, os.path.join(trained_models_path, file_name))

 else:
  checkpoints_directory = f"{latest_training_directory}/checkpoints/trainstep_checkpoints"
  file_paths = glob.glob(f"{checkpoints_directory}/*")

  for i, original_file_name in enumerate(file_paths):
   # Remove the "epoch=000000-step=0000" text
   steps = re.sub(checkpoints_directory + "/epoch=\d{6}-step=0*", "", original_file_name)

   # Remove the .ckpt
   steps = steps.replace(".ckpt", "")

   # Setup the filename
   file_name = create_checkpoint_file_name(
    config_parsed = config_parsed,
    date = date_string[-1],
    training_images_count = training_images_count,
    training_steps = steps,
   )

   # Move the checkpoint
   shutil.move(original_file_name, os.path.join(trained_models_path, file_name))

 print("✅ Download your trained model(s) from the 'trained_models' folder and use in your favorite Stable Diffusion repo!")

# Optional - Upload to google drive
* run the following commands in a new `terminal` in the `Dreambooth-Stable-Diffusion` directory
* `chmod +x ./gdrive`
* `./gdrive about`
* `paste your token here after navigating to the link`
* `./gdrive upload trained_models/{file_name.ckpt}`

# Big Important Note!

The way to use your token is `<token> <class>` ie `joepenna person` and not just `joepenna`